In [1]:
import numpy as np
import pandas as pd
import numpy as np
import ollama

import warnings
warnings.filterwarnings("ignore")

In [2]:
output_path = 'output/models/'

In [3]:
movies = pd.read_csv('Nikolay/dataset/ml-latest-small/movies.csv')
movies = pd.read_csv('Aleksis/movies_with_abstracts.csv')
ratings = pd.read_csv('Nikolay/dataset/ml-latest-small/ratings.csv')

directed_by_path = 'Nikolay/dataset/ml-latest-small/directed_by.csv'
stars_path = 'Nikolay/dataset/ml-latest-small/stars.csv'
produced_by_path = 'Nikolay/dataset/ml-latest-small/produced_by.csv'
produced_in_path = 'Nikolay/dataset/ml-latest-small/produced_in.csv'

directed_by_df = pd.read_csv(directed_by_path)[["movieId", "director"]]
stars_df = pd.read_csv(stars_path)[["movieId", "castMember"]]
produced_by_df = pd.read_csv(produced_by_path)[["movieId", "company"]]
produced_in_df = pd.read_csv(produced_in_path)[["movieId", "country"]]

In [4]:
movies['abstracts']

0                                                     NaN
1       Jumanji is a 1995 American fantasy adventure f...
2       Grumpier Old Men is a 1995 American romantic c...
3                                                     NaN
4                                                     NaN
                              ...                        
9737    The Butler Bulldogs men's basketball team repr...
9738    No Game No Life (Japanese: ノーゲーム・ノーライフ, Hepbur...
9739    Flint is a sedimentary cryptocrystalline form ...
9740    Bungo Stray Dogs (Japanese: 文豪ストレイドッグス, Hepbur...
9741    Andrew Dice Clay (born Andrew Clay Silverstein...
Name: abstracts, Length: 9742, dtype: object

In [5]:
movies[movies['abstracts'].isna()]['genres']

0       Adventure|Animation|Children|Comedy|Fantasy
3                              Comedy|Drama|Romance
4                                            Comedy
7                                Adventure|Children
9                         Action|Adventure|Thriller
                           ...                     
9720                      Action|Adventure|Thriller
9725                                  Drama|Romance
9727                                          Drama
9728                                          Drama
9733                                Animation|Drama
Name: genres, Length: 3040, dtype: object

In [6]:
movies.loc[movies['abstracts'].isna(), 'abstracts'] = movies[movies['abstracts'].isna()]['genres']

In [7]:
import pickle
with open('Aleksis/movie2abstract_embed.pickle', 'rb') as f:
    movie2abstract_embed = pickle.load(f)

In [8]:
movies

,movieId,title,genres,abstracts,abstracts_count
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,Adventure|Animation|Children|Comedy|Fantasy,0
1,2,Jumanji (1995),Adventure|Children|Fantasy,Jumanji is a 1995 American fantasy adventure f...,1
2,3,Grumpier Old Men (1995),Comedy|Romance,Grumpier Old Men is a 1995 American romantic c...,1
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,Comedy|Drama|Romance,0
4,5,Father of the Bride Part II (1995),Comedy,Comedy,0
...,...,...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy,The Butler Bulldogs men's basketball team repr...,2
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy,"No Game No Life (Japanese: ノーゲーム・ノーライフ, Hepbur...",1
9739,193585,Flint (2017),Drama,Flint is a sedimentary cryptocrystalline form ...,1
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation,"Bungo Stray Dogs (Japanese: 文豪ストレイドッグス, Hepbur...",2


In [9]:
movies_with_abstracts = movies[movies['abstracts_count'] > 0]

In [10]:
# movie2abstract_embed[1]

In [11]:
len(movies_with_abstracts)

6702

In [12]:
import pickle
with open('Aleksis/movie2abstract_embed.pickle', 'rb') as f:
    movie2abstract_embed = pickle.load(f)

In [13]:
ratings.loc[[1,2, 3]]

,userId,movieId,rating,timestamp
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815


In [14]:
ratings_with_abstracts_idx = []
for i in range(len(ratings)):
    if ratings.loc[i]['movieId'] in movies_with_abstracts['movieId']:
        ratings_with_abstracts_idx.append(i)

In [15]:
ratings.loc[[i for i in range(len(ratings))]]

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [16]:
ratings_with_abstracts = ratings.loc[ratings_with_abstracts_idx]

In [17]:
positive_ratings = ratings_with_abstracts[ratings_with_abstracts['rating'] >= 4]

In [18]:
user_ratings_count = positive_ratings['userId'].value_counts(ascending=True)

In [19]:
users = positive_ratings['userId'].unique()

In [20]:
users_with_enough_ratings = list(filter(lambda x: user_ratings_count[x] >= 100, users))

In [21]:
len(users_with_enough_ratings)

73

In [22]:
userId = np.random.choice(users_with_enough_ratings)
userId

356

In [23]:
rated_movies_ids = positive_ratings[positive_ratings['userId'] == userId]['movieId']

In [24]:
movieId = np.random.choice(rated_movies_ids)
movieId

5618

In [25]:
rated_movies = movies_with_abstracts.loc[rated_movies_ids]
rated_movies

,movieId,title,genres,abstracts,abstracts_count
62,70,From Dusk Till Dawn (1996),Action|Comedy|Horror|Thriller,From Dusk till Dawn is a 1996 American action ...,1
141,169,Free Willy 2: The Adventure Home (1995),Adventure|Children|Drama,Free Willy 2: The Adventure Home is a 1995 Ame...,1
180,212,Bushwhacked (1995),Adventure|Comedy|Crime|Mystery,Bushwhacked is a 1995 American adventure comed...,1
208,242,Farinelli: il castrato (1994),Drama|Musical,Farinelli (Italian pronunciation: [fariˈnɛlli]...,2
266,306,Three Colors: Red (Trois couleurs: Rouge) (1994),Drama,The Three Colours trilogy (French: Trois coule...,1
...,...,...,...,...,...
8665,121171,Red Army (2014),Documentary,The Workers' and Peasants' Red Army (Russian: ...,1
8784,129333,Julia (2014),Horror|Thriller,"Julia the Elder (30 October 39 BC – AD 14), kn...",1
8796,130073,Cinderella (2015),Children|Drama|Fantasy|Romance,"""Cinderella"", or ""The Little Glass Slipper"", i...",1
8910,135436,The Secret Life of Pets (2016),Animation|Comedy,Secrecy is the practice of hiding information ...,1


In [26]:
!pip install sentence_transformers

In [27]:
from sentence_transformers import SentenceTransformer

sentence_transformer = SentenceTransformer("all-mpnet-base-v2")

In [28]:
abstracts_embeddings = sentence_transformer.encode(list(rated_movies['abstracts']))
target_movie_embedding = sentence_transformer.encode(list([rated_movies.loc[movieId]['abstracts']]))

In [29]:
len(abstracts_embeddings), len(target_movie_embedding)

(113, 1)

In [30]:
abstracts_embeddings

array([[ 0.03470659, -0.00895606,  0.02537801, ..., -0.03682916,
         0.02896097, -0.02092162],
       [ 0.03649786,  0.03177823,  0.00804228, ..., -0.00590595,
         0.03050319,  0.04792884],
       [ 0.04168453,  0.00244346,  0.02535284, ..., -0.03092151,
        -0.01379859, -0.03257846],
       ...,
       [ 0.07607358,  0.00966167, -0.01822169, ..., -0.02135365,
         0.04995064, -0.03879547],
       [ 0.03574057,  0.02723282, -0.00641413, ...,  0.04904644,
         0.03236689,  0.01710298],
       [-0.01595804,  0.04740734, -0.01552411, ...,  0.02086556,
         0.05495937,  0.00934624]], dtype=float32)

In [31]:
from sklearn.metrics.pairwise import cosine_similarity

In [32]:
cosine_similarity(target_movie_embedding, abstracts_embeddings).shape

(1, 113)

In [33]:
similarities = cosine_similarity(target_movie_embedding, abstracts_embeddings)

In [34]:
movieId

5618

In [35]:
np.argsort(similarities)[0][-1]

91

In [36]:
list(rated_movies['abstracts'])[np.argsort(similarities)[0][-1]]

'Vidocq (North American DVD title: Dark Portals: The Chronicles of Vidocq) is a 2001 mystery film, directed by Pitof, starring Gérard Depardieu as historical figure Eugène François Vidocq pursuing a supernatural serial killer. It is notable as being the first major fantasy film to be released that was shot entirely with digital cinematography, using a Sony HDW-F900 CineAlta camera. The Finnish cello metal band Apocalyptica used clips from the film in the music video for their Cult track, "Hope Vol. 2" with Matthias Sayer of Farmer Boys providing extra recorded vocals, which served as the film\'s ending theme.<sep>Vidocq (North American DVD title: Dark Portals: The Chronicles of Vidocq) is a 2001 mystery film, directed by Pitof, starring Gérard Depardieu as historical figure Eugène François Vidocq pursuing a supernatural serial killer. It is notable as being the first major fantasy film to be released that was shot entirely with digital cinematography, using a Sony HDW-F900 CineAlta cam

In [37]:
rated_movies.loc[movieId]['abstracts']

'Vidocq (North American DVD title: Dark Portals: The Chronicles of Vidocq) is a 2001 mystery film, directed by Pitof, starring Gérard Depardieu as historical figure Eugène François Vidocq pursuing a supernatural serial killer. It is notable as being the first major fantasy film to be released that was shot entirely with digital cinematography, using a Sony HDW-F900 CineAlta camera. The Finnish cello metal band Apocalyptica used clips from the film in the music video for their Cult track, "Hope Vol. 2" with Matthias Sayer of Farmer Boys providing extra recorded vocals, which served as the film\'s ending theme.<sep>Vidocq (North American DVD title: Dark Portals: The Chronicles of Vidocq) is a 2001 mystery film, directed by Pitof, starring Gérard Depardieu as historical figure Eugène François Vidocq pursuing a supernatural serial killer. It is notable as being the first major fantasy film to be released that was shot entirely with digital cinematography, using a Sony HDW-F900 CineAlta cam

In [38]:
most_similar_movies_idx = np.argsort(similarities)[0][-4:]
most_similar_movies_idx

array([103,  11,   0,  91], dtype=int64)

In [39]:
most_similar_movies = rated_movies.iloc[most_similar_movies_idx]

In [40]:
most_similar_movies

,movieId,title,genres,abstracts,abstracts_count
7352,78349,Exam (2009),Mystery|Thriller,Exam is a 2009 British psychological thriller ...,1
543,640,Diabolique (1996),Drama|Thriller,"Les Diaboliques (French: [le djabɔlik], releas...",1
62,70,From Dusk Till Dawn (1996),Action|Comedy|Horror|Thriller,From Dusk till Dawn is a 1996 American action ...,1
5618,27134,Dark Portals: The Chronicles of Vidocq (Vidoc...,Action|Crime|Fantasy,Vidocq (North American DVD title: Dark Portals...,2


In [41]:
prompt = '''
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a helpful AI assistant that that recommends movies to users.
You will be given the names and short descriptions of 3 movies a user has liked.
And after that you will get the name and description of the movie you need to recommend.
Based on the descriptions of the movies the user has already liked and the movie you are 
supposed to recommend write a enticing recommendation on why the user may like the movie.
{movies_example}
<|eot_id|>
<|start_header_id|>user<|end_header_id|>
<|eot_id|><|start_header_id|>assistant<|end_header_id|>
'''

In [42]:
movies_example = ''
for i in range(len(most_similar_movies) - 1):
    movies_example += f'Movie name {i + 1}: {most_similar_movies.iloc[i]["title"]}\n'
    movies_example += f'Movie name {i + 1}: {most_similar_movies.iloc[i]["abstracts"]}\n'

movies_example += f'\nName of movie you will recommend: {most_similar_movies.iloc[-1]["title"]}\n'
movies_example += f'Description of movie you will recommend: {most_similar_movies.iloc[-1]["abstracts"]}\n'

In [43]:
print(movies_example)

Movie name 1: Exam (2009)
Movie name 1: Exam is a 2009 British psychological thriller film written by Simon Garrity and Stuart Hazeldine, directed by Hazeldine, and starring Colin Salmon, Chris Carey, Jimi Mistry, Luke Mably, Gemma Chan, Chukwudi Iwuji, John Lloyd Fillingham, Pollyanna McIntosh, Adar Beck and Nathalie Cox.
Movie name 2: Diabolique (1996)
Movie name 2: Les Diaboliques (French: [le djabɔlik], released as Diabolique in the United States and variously translated as The Devils or The Fiends) is a 1955 French psychological horror thriller film directed by Henri-Georges Clouzot, starring Simone Signoret, Véra Clouzot, Paul Meurisse and Charles Vanel. It is based on the novel She Who Was No More (Celle qui n'était plus) by Pierre Boileau and Thomas Narcejac. The story blends elements of thriller and horror, with the plot focusing on a woman and her husband's mistress who conspire to murder the man. The film was the 10th-highest grossing film of the year in France, with a total

In [44]:
print(prompt.format(movies_example=movies_example))


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a helpful AI assistant that that recommends movies to users.
You will be given the names and short descriptions of 3 movies a user has liked.
And after that you will get the name and description of the movie you need to recommend.
Based on the descriptions of the movies the user has already liked and the movie you are 
supposed to recommend write a enticing recommendation on why the user may like the movie.
Movie name 1: Exam (2009)
Movie name 1: Exam is a 2009 British psychological thriller film written by Simon Garrity and Stuart Hazeldine, directed by Hazeldine, and starring Colin Salmon, Chris Carey, Jimi Mistry, Luke Mably, Gemma Chan, Chukwudi Iwuji, John Lloyd Fillingham, Pollyanna McIntosh, Adar Beck and Nathalie Cox.
Movie name 2: Diabolique (1996)
Movie name 2: Les Diaboliques (French: [le djabɔlik], released as Diabolique in the United States and variously translated as The Devils or The Fiends) is a 1955 

In [45]:
output = ollama.generate(model='llama3', prompt=prompt.format(movies_example=movies_example), options=dict(temperature=0.3))

In [46]:
response_with_abstracts = output['response']

In [47]:
print(output['response'])

Based on the movies you've liked (Exam, Diabolique, and From Dusk Till Dawn), I think you might enjoy "Dark Portals: The Chronicles of Vidocq" for several reasons:

1. Psychological Thriller Elements: Like Exam and Diabolique, Vidocq has a strong focus on mystery and suspense. You'll likely appreciate the intricate plot twists and the cat-and-mouse game between Vidocq and the supernatural serial killer.
2. Dark and Unsettling Atmosphere: From Dusk Till Dawn is known for its dark humor and over-the-top violence. While Vidocq doesn't have the same level of gore, it does share a similar sense of unease and foreboding. The film's use of digital cinematography creates a unique, eerie atmosphere that will likely appeal to fans of darker, more suspenseful movies.
3. Complex Characters: All three of your liked movies feature complex, morally ambiguous characters. Vidocq is no exception, with its protagonist being a flawed and intriguing figure. You'll likely appreciate the character-driven sto

In [48]:
prompt = '''
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a helpful AI assistant that that recommends movies to users.
You will be given the names of 3 movies a user has liked.
And after that you will get the name of the movie you need to recommend.
Based on the movies the user has already liked and the movie you are
supposed to recommend write a enticing recommendation on why the user may like the movie.
{movies_example}
<|eot_id|>
<|start_header_id|>user<|end_header_id|>
<|eot_id|><|start_header_id|>assistant<|end_header_id|>
'''

In [49]:
movies_example = ''
for i in range(len(most_similar_movies) - 1):
    movies_example += f'Movie name {i + 1}: {most_similar_movies.iloc[i]["title"]}\n'

movies_example += f'\nName of movie you will recommend: {most_similar_movies.iloc[-1]["title"]}\n'

In [50]:
output = ollama.generate(model='llama3', prompt=prompt.format(movies_example=movies_example), options=dict(temperature=0.3))

In [51]:
response = output['response']

In [52]:
print(output['response'])

I'm excited to make a recommendation for you!

Based on the movies you've liked, such as "Exam" (a psychological thriller with a unique setting), "Diabolique" (a suspenseful and dark mystery), and "From Dusk Till Dawn" (a gritty and intense crime thriller), I think you might enjoy "Dark Portals: The Chronicles of Vidocq" (2001).

Here's why:

Like "Exam", "Vidocq" features a complex, puzzle-like plot that keeps you guessing until the very end. You'll be drawn into the world of Inspector Vidocq, a charismatic and cunning detective who must unravel a web of deceit and corruption in 19th-century Paris.

The film's blend of mystery, suspense, and intrigue is reminiscent of "Diabolique", which also kept audiences on the edge of their seats with its clever twists and turns. And if you enjoyed the gritty, crime-ridden atmosphere of "From Dusk Till Dawn", you'll appreciate the dark, gritty tone of "Vidocq" as he navigates the mean streets of Paris.

Overall, I think "Dark Portals: The Chronicl

In [53]:
prompt = '''
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You will be given two different movie recommendations they are both for 
the same user and movie. Evaluate which of the two is better and more likely
to entice the user to watch the movie. Not that the recommendation are based
on movies the user has already liked.
First recommendation:
{first}
Second recommendation:
{second}

Now evaluate which of the two recommendations is better.
<|eot_id|>
<|start_header_id|>user<|end_header_id|>
<|eot_id|><|start_header_id|>assistant<|end_header_id|>
'''

In [54]:
output = ollama.generate(model='llama3', prompt=prompt.format(first=response_with_abstracts, second=response), options=dict(temperature=0.3))

In [55]:
print(output['response'])

After evaluating both recommendations, I think the first recommendation stands out as more effective in enticing the user to watch the movie. Here's why:

1. The first recommendation provides specific reasons why the user might enjoy the movie, including psychological thriller elements, dark and unsettling atmosphere, and complex characters. This shows that the recommender has taken the time to understand the user's tastes and preferences.
2. The language used in the first recommendation is more engaging and descriptive, painting a vivid picture of the movie's tone and themes. For example, phrases like "intricate plot twists", "unique, eerie atmosphere", and "flawed and intriguing figure" create a sense of excitement and curiosity.
3. The first recommendation also provides a clear connection between the user's liked movies (Exam, Diabolique, and From Dusk Till Dawn) and why they might enjoy Vidocq. This helps to establish a sense of familiarity and relevance.

In contrast, the second r